
# **Asignatura de Big Data Analytics**

Prof. José M. Luna

jmluna@uco.es

---

En primer lugar, debemos instalar todas las dependencias necesarias para poder ejecutar el algoritmo. Así, tenemos que instalar Java 8, Spark 3.0 con Hadoop 3.2, y findspark para alojar spark en el sistema (acceder a Spark desde un Notebook).

In [ ]:
# Instalacion de java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalacion de spark
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# Descomprimimos el fichero de spark
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# Instalamos findspark
!pip install -q findspark

# Establecemos las variables de entorno para Java y Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

Ejecutamos una sesion local de spark para comprobar que esta funcionando.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Importamos el conjunto de datos con el que queremos trabajar y comprobamos que se ha subido correctamente

In [ ]:
from google.colab import files
uploaded = files.upload()

!ls

Saving market.csv to market.csv
market.csv   spark-3.0.0-bin-hadoop3.2
sample_data  spark-3.0.0-bin-hadoop3.2.tgz


Leemos el dataset con el que queremos trabajar, en este caso vamos a trabajar con el dataset de BostonHousing



In [ ]:
dataset = spark.read.csv('market.csv',inferSchema=True, header =True)

Mostramos los atributos que tiene el dataset y su tipo

In [ ]:
dataset.printSchema()
dataset.show()

root
 |-- SalesDate: string (nullable = true)
 |-- SalesValue: double (nullable = true)
 |-- SalesAmount: integer (nullable = true)
 |-- Customer: integer (nullable = true)
 |-- SalesTransactionID: integer (nullable = true)
 |-- SalesItem: integer (nullable = true)

+-------------------+----------+-----------+--------+------------------+---------+
|          SalesDate|SalesValue|SalesAmount|Customer|SalesTransactionID|SalesItem|
+-------------------+----------+-----------+--------+------------------+---------+
|2018-09-28 00:00:00|    8280.0|         10|       0|                 0|        0|
|2018-09-28 00:00:00|    7452.0|         10|       0|                 0|        0|
|2019-04-23 00:00:00|   21114.0|         30|       0|                 1|        0|
|2019-04-23 00:00:00|    7038.0|         10|       0|                 1|        1|
|2019-04-23 00:00:00|    7000.0|          2|       0|                 1|        2|
|2019-02-08 00:00:00|     745.2|          1|       0|                

Reordenamos el dataset, considerando sólo los ID de las transacciones y los items vendidos en cada transaccion. Ademas, una misma transaccion no puede tener el mismo item dos o mas veces

In [ ]:
from pyspark.sql import functions
from pyspark.ml.fpm import FPGrowth
import pandas

basketdata = dataset.dropDuplicates(['SalesTransactionID', 'SalesItem']).sort('SalesTransactionID')
basketdata = basketdata.groupBy("SalesTransactionID").agg(functions.collect_list("SalesItem")).sort('SalesTransactionID')

basketdata.printSchema()
basketdata.show()

root
 |-- SalesTransactionID: integer (nullable = true)
 |-- collect_list(SalesItem): array (nullable = true)
 |    |-- element: integer (containsNull = false)

+------------------+-----------------------+
|SalesTransactionID|collect_list(SalesItem)|
+------------------+-----------------------+
|                 0|                    [0]|
|                 1|              [0, 2, 1]|
|                 2|                    [1]|
|                 3|                    [0]|
|                 4|                    [0]|
|                 5|                    [0]|
|                 6|                    [2]|
|                 7|                    [2]|
|                 8|                    [0]|
|                10|                 [1, 0]|
|                11|                    [0]|
|                12|                 [4, 3]|
|                13|                    [1]|
|                15|              [2, 4, 3]|
|                16|           [3, 2, 0, 1]|
|                17|         

Ejecutamos el algoritmo FPGrowth sobre el conjunto de datos transformado, considerando unos valores minimos de soporte y confianza.

In [ ]:
#FP-Growth algorithm
fpGrowth = FPGrowth(itemsCol="collect_list(SalesItem)", minSupport=0.2, minConfidence=0.3)
itemsets = fpGrowth.fit(basketdata)
# Mostramos los items frecuentes
itemsets.freqItemsets.show()
# Mostramos las reglas de asociacion
itemsets.associationRules.show()

+---------+----+
|    items|freq|
+---------+----+
|      [0]|  11|
|      [2]|   9|
|   [2, 0]|   4|
|      [1]|   7|
|   [1, 2]|   4|
|[1, 2, 0]|   4|
|   [1, 0]|   5|
|      [3]|   5|
|   [3, 2]|   4|
|      [4]|   4|
|   [4, 3]|   4|
+---------+----+

+----------+----------+-------------------+------------------+
|antecedent|consequent|         confidence|              lift|
+----------+----------+-------------------+------------------+
|    [1, 0]|       [2]|                0.8| 1.688888888888889|
|       [4]|       [3]|                1.0|3.8000000000000003|
|       [3]|       [2]|                0.8| 1.688888888888889|
|       [3]|       [4]|                0.8|3.8000000000000003|
|       [1]|       [2]| 0.5714285714285714|1.2063492063492063|
|       [1]|       [0]| 0.7142857142857143|1.2337662337662338|
|    [2, 0]|       [1]|                1.0|2.7142857142857144|
|       [2]|       [0]| 0.4444444444444444|0.7676767676767676|
|       [2]|       [1]| 0.4444444444444444|1.206349